In [21]:
# import libraries
import requests
import pandas as pd
import os
import psycopg2
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [22]:
# extract F1 data from API
drivers_api = requests.get('https://api.openf1.org/v1/drivers').json()
meetings_api = requests.get('https://api.openf1.org/v1/meetings').json()
weather_api = requests.get('https://api.openf1.org/v1/weather').json()

# convert to DataFrames
drivers_df = pd.DataFrame(drivers_api)
meetings_df = pd.DataFrame(meetings_api)
weather_df = pd.DataFrame(weather_api)

# join DataFrames
f1_df = drivers_df.merge(meetings_df, on='meeting_key', how='inner').merge(weather_df, on=['meeting_key', 'session_key'], how='inner')


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# preview columns
f1_df.columns

Index(['driver_number', 'broadcast_name', 'full_name', 'name_acronym',
       'team_name', 'team_colour', 'first_name', 'last_name', 'headshot_url',
       'country_code_x', 'session_key', 'meeting_key', 'meeting_name',
       'meeting_official_name', 'location', 'country_key', 'country_code_y',
       'country_name', 'circuit_key', 'circuit_short_name', 'date_start',
       'gmt_offset', 'year', 'meeting_code', 'air_temperature', 'humidity',
       'pressure', 'rainfall', 'track_temperature', 'wind_direction',
       'wind_speed', 'date'],
      dtype='object')

In [ ]:
#  connect to Amazon RDS and create a table
conn = psycopg2.connect(
            host = 'postgresql.c7cao8o2cjkw.eu-west-2.rds.amazonaws.com',
            dbname = 'f1',
            user = 'postgres',
            password = 'mydb123$$',
            port = '5432'
        )
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS f1_data (
    meeting_key INT,
    session_key INT,
    driver_number INT,
    full_name TEXT,
    team_name TEXT,
    team_colour TEXT,
    headshot_url TEXT,
    country_code TEXT,
    meeting_name TEXT,
    meeting_official_name TEXT,
    location TEXT,
    country_name TEXT,
    circuit_short_name TEXT,
    date_start DATE,
    gmt_offset TEXT,
    year INT,
    meeting_code TEXT,
    air_temperature FLOAT,
    humidity FLOAT,
    pressure FLOAT,
    rainfall FLOAT,
    track_temperature FLOAT,
    wind_direction TEXT,
    wind_speed FLOAT,
    date_weather TIMESTAMP
);
""")

# Preview table
query = "SELECT * FROM f1_data LIMIT 10;"
df = pd.read_sql(query, conn)
print(df.head())

   meeting_key  session_key  driver_number        full_name        team_name  \
0         1217         9144             31     Esteban OCON           Alpine   
1         1250         9640             14  Fernando ALONSO     Aston Martin   
2         1255         9993             14  Fernando ALONSO     Aston Martin   
3         1208         9072             24      ZHOU Guanyu             None   
4         1220         9168              1   Max VERSTAPPEN  Red Bull Racing   

  team_colour                                       headshot_url country_code  \
0      2293D1  https://www.formula1.com/content/dam/fom-websi...          FRA   
1      229971  https://media.formula1.com/d_driver_fallback_i...          ESP   
2      229971  https://media.formula1.com/d_driver_fallback_i...          ESP   
3        None                                               None         None   
4      3671C6  https://www.formula1.com/content/dam/fom-websi...          NED   

           meeting_name         

/var/folders/rf/_hq8mr4j4kj5ddz7rqkyj9nw0000gn/T/ipykernel_22373/2161034371.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
# load API data to RDS
cur.execute("""
        TRUNCATE TABLE f1_data;
""")
for index, row in f1_df.sample(10000).iterrows():
    cur.execute("""
        INSERT INTO f1_data (
            meeting_key, session_key, driver_number, full_name, team_name, team_colour,
            headshot_url, country_code, meeting_name, meeting_official_name, location,
            country_name, circuit_short_name, date_start, gmt_offset, year, meeting_code,
            air_temperature, humidity, pressure, rainfall, track_temperature, wind_direction,
            wind_speed, date_weather
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['meeting_key'],
        row['session_key'],
        row['driver_number'],
        row['full_name'],
        row['team_name'],
        row['team_colour'],
        row['headshot_url'],
        row['country_code_x'],
        row['meeting_name'],
        row['meeting_official_name'],
        row['location'],
        row['country_name'],
        row['circuit_short_name'],
        row['date_start'],
        row['gmt_offset'],
        row['year'],
        row['meeting_code'],
        row['air_temperature'],
        row['humidity'],
        row['pressure'],
        row['rainfall'],
        row['track_temperature'],
        row['wind_direction'],
        row['wind_speed'],
        row['date']
    ))


# Preview table
query = "SELECT * FROM f1_data LIMIT 10;"
df = pd.read_sql(query, conn)
print(df.head())

   meeting_key  session_key  driver_number        full_name        team_name  \
0         1238         9532             55     Carlos SAINZ          Ferrari   
1         1210         9089             16  Charles LECLERC          Ferrari   
2         1217         9149             11     Sergio PEREZ  Red Bull Racing   
3         1208         9071             10     Pierre GASLY           Alpine   
4         1141         7767              4     Lando NORRIS          McLaren   

  team_colour                                       headshot_url country_code  \
0      E80020  https://media.formula1.com/d_driver_fallback_i...          ESP   
1      F91536  https://www.formula1.com/content/dam/fom-websi...          MON   
2      3671C6  https://www.formula1.com/content/dam/fom-websi...          MEX   
3      2293D1  https://www.formula1.com/content/dam/fom-websi...          FRA   
4      F58020  https://www.formula1.com/content/dam/fom-websi...          GBR   

         meeting_name           

/var/folders/rf/_hq8mr4j4kj5ddz7rqkyj9nw0000gn/T/ipykernel_22373/2535988249.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
# create mart
cur.execute("""             
    DROP TABLE IF EXISTS f1_mart;""")
cur.execute("""
    CREATE TABLE f1_mart AS
        WITH meeting_session_agg AS (
        SELECT
            meeting_key,
            AVG(air_temperature) AS avg_air_temperature,
            AVG(humidity) AS avg_humidity,
            AVG(pressure) AS avg_pressure,
            SUM(rainfall) AS total_rainfall,
            AVG(track_temperature) AS avg_track_temperature,
            AVG(wind_speed) AS avg_wind_speed,
            MIN(date_weather) AS first_recorded_weather
        FROM
            f1_data
        GROUP BY
            meeting_key
)             
        SELECT 
            f.meeting_key,
            f.session_key,
            f.full_name,
            MAX(f.driver_number) AS driver_number,
            COUNT(f.headshot_url) AS headshot_count,
            MAX(f.team_name) AS team_name,
            MAX(f.team_colour) AS team_colour,
            MAX(f.country_code) AS driver_country,
            MAX(f.meeting_name) AS meeting_name,
            MAX(f.meeting_official_name) AS meeting_official_name,
            MAX(f.location) AS location,
            MAX(f.country_name) AS country_name,
            MAX(f.circuit_short_name) AS circuit_short_name,
            MIN(f.date_start) AS date_start,
            MAX(f.gmt_offset) AS gmt_offset,
            MAX(f.year) AS year,
            MAX(f.meeting_code) AS meeting_code,
            m.avg_air_temperature,
            m.avg_humidity,
            m.avg_pressure,
            m.total_rainfall,
            m.avg_track_temperature,
            m.avg_wind_speed,
            m.first_recorded_weather

        FROM 
            f1_data AS f
        LEFT JOIN 
            meeting_session_agg AS m
        ON 
            f.meeting_key = m.meeting_key

        GROUP BY 
            f.meeting_key, f.session_key, f.full_name, 
            m.avg_air_temperature, m.avg_humidity, m.avg_pressure, 
            m.total_rainfall, m.avg_track_temperature, m.avg_wind_speed, m.first_recorded_weather

        ORDER BY 
            f.meeting_key, f.session_key;
        ;""")
conn.commit()

# Preview table
query = "SELECT * FROM f1_mart;"
f1_mart_df = pd.read_sql(query, conn)
print(f1_mart_df.head())

/var/folders/rf/_hq8mr4j4kj5ddz7rqkyj9nw0000gn/T/ipykernel_22373/896052058.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  f1_mart_df = pd.read_sql(query, conn)


   meeting_key  session_key        full_name  driver_number  headshot_count  \
0         1140         7763     Carlos SAINZ             55               1   
1         1140         7763  Charles LECLERC             16               4   
2         1140         7763     Esteban OCON             31               3   
3         1140         7763  Fernando ALONSO             14               1   
4         1140         7763   George RUSSELL             63               2   

      team_name team_colour driver_country        meeting_name  \
0       Ferrari      F91536            ESP  Pre-Season Testing   
1       Ferrari      F91536            MON  Pre-Season Testing   
2        Alpine      2293D1            FRA  Pre-Season Testing   
3  Aston Martin      358C75            ESP  Pre-Season Testing   
4      Mercedes      6CD3BF            GBR  Pre-Season Testing   

                      meeting_official_name  ... gmt_offset  year  \
0  FORMULA 1 ARAMCO PRE-SEASON TESTING 2023  ...   03:00:00

In [ ]:
# create a Google Sheets spreadsheet with f1 data
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("etl-f1-de33e29ab6c8.json", scope)
client = gspread.authorize(creds)

# Open target Google Sheet
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1_ITQ17SUBtdiacBrHBdl8H84pn9We1OSr2_9xZHXdeA")
sheet = spreadsheet.sheet1

# Clear existing data
sheet.clear()

# convert dates to strings
f1_mart_df['date_start'] = f1_mart_df['date_start'].astype(str)
f1_mart_df['first_recorded_weather'] = f1_mart_df['first_recorded_weather'].astype(str)

# Write headers
sheet.insert_row(f1_mart_df.columns.tolist(), index=1)

# Write data rows
sheet.insert_rows(f1_mart_df.values.tolist(), row=2)

{'spreadsheetId': '1_ITQ17SUBtdiacBrHBdl8H84pn9We1OSr2_9xZHXdeA',
 'updates': {'spreadsheetId': '1_ITQ17SUBtdiacBrHBdl8H84pn9We1OSr2_9xZHXdeA',
  'updatedRange': 'Sheet1!A2:X1567',
  'updatedRows': 1566,
  'updatedColumns': 24,
  'updatedCells': 36706}}